In [1]:
import pandas as pd
import numpy as np

In [2]:
disasters = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/disasters_2016.csv')
codes = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/country_codes.csv')
antirating_1 = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/rating_donation_v0.csv')
donations = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/data_merged_2006-2022.csv')
conflicts = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/conflicts_2016.csv') ## мб плохой источник
gini = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/gini_worldbank.csv',skiprows=3)
gdp = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/gdp_worldbank.csv',skiprows=4).fillna(0)
gdp_2016 = gdp[['Country Name','Country Code','Indicator Name','Indicator Code','2016']]

# conflicts_ = pd.read_csv(r"C:\Users\filim\Downloads\time_series_short_new_0.xls")

In [5]:
pd.read_csv('data_3sided_model_(coflict_disaster_gdp)/table_long.csv')

,Code,Prevalence of undernourishment (% of population),People using at least basic drinking water services (% of population),Annual CO2 emissions (per capita),People using at least basic sanitation services (% of population),Primary energy consumption per capita (kWh/person)
0,AFG,22.200001,64.054886,0.1906,44.201218,973.857361
1,CAF,48.599998,41.276642,0.0436,15.588242,332.125702
2,COL,6.800000,96.616966,1.9355,89.964920,10472.013672
3,COD,39.900002,43.355839,0.0256,16.983427,428.116852
4,ETH,13.800000,43.607079,0.1385,7.779459,820.405823
...,...,...,...,...,...,...
8,MMR,8.400000,75.566376,0.4795,71.735939,2434.731689
9,PAK,12.400000,89.596367,1.0166,61.064262,4369.207520
10,SOM,57.799999,50.274597,0.0449,35.064613,255.680084
11,SDN,11.800000,60.215885,0.4846,36.485626,2651.733887


In [4]:
pd.read_csv('data_3sided_model_(coflict_disaster_gdp)/gdp_2016.csv')

,Country Name,Country Code,2016
0,Aruba,ABW,2.983799e+09
1,Africa Eastern and Southern,AFE,8.823551e+11
2,Afghanistan,AFG,1.811656e+10
3,Africa Western and Central,AFW,6.905464e+11
4,Angola,AGO,4.984049e+10
...,...,...,...
261,Kosovo,XKX,6.682833e+09
262,"Yemen, Rep.",YEM,3.131783e+10
263,South Africa,ZAF,3.235855e+11
264,Zambia,ZMB,2.095841e+10
